## Getting Started with Chains

Langchain의 핵심은 '체인(Chains)'이다. 말 그대로 '요소(Components)'의 연결을 말하며 순서대로 요소들이 실행이 된다.

체인 중에서 가장 간단한 형태가 <b>LLMChain</b>이다. LLMChain은 인풋을 받아서 chain의 첫번째 파라미터인 <b>PromptTemplate</b>에 전달을 해서 인풋을 특정 프롬프트로 포맷을 만든다. 포맷이 된 프롬프트는 체인에서 그 다음 요소인 LLM에게 전달이 되는 것이다.

먼저 이번 챕터에서 사용하게 될 라이브러리들을 먼저 import하는 것으로 코드를 시작하자.

In [4]:
import inspect
import re

from dotenv import load_dotenv
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain, LLMMathChain, TransformChain, SequentialChain
from langchain.callbacks import get_openai_callback


이 챕터에서는 OpenAI LLM을 이용하기 때문에 OpenAI API key를 획득했다고 가정한다.

In [7]:
load_dotenv()

llm = OpenAI()
llm.temperature=0

다음으로 간단한 유틸리티 함수를 하나 만들 것이다. 이 함수는 우리가 OpenAI의 API를 사용하면서 얼마나 많은 토큰을 소비하는 지를 알려준다. 이런 유틸리티를 만들어서 소비하는 토큰의 갯수를 확인하는 것은 꽤 중요한 일이다. 우리가 복잡한 기능을 하는 툴,에이전트를 만들게 되면 API를 여러번 호출하게 될텐데 이 때 우리가 너무 많은 토큰을 소비해서 비용이 많이 나오지는 않은 지에 대해서 계속해서 확인을 하고 제어를 하는 것이 필요하다.

In [8]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'전체 사용 토큰 : {cb.total_tokens}')
    return result


### 그래서 대체 체인이 뭐지?

<b>정의</b>: 프롬프트, llm, 유틸리티와 같은 기초 요소(primitives)들을 연결(link)한 것, 혹은 이런 연결로 만들어 진 체인을 다시 연결한 것들이다.

연결이라고 함은 인풋을 기초 요소들의 조합을 사용해서 프로세싱 하는 파이프라인을 이야기 한다. 직관적으로 이야기 하면 체인이라는 것은 주어진 인풋에 대해서 결과를 내보내기 위해 수행하는 작업(operation) 집합인데, 이 집합이 '단계적(step)'으로 수행이 된다고 할 수 있다. 작업(operation)은 단순히 프롬프트를 만들어서 LLM에게 전달해 버리는 것일 수도 있고 혹은 텍스트에 대해서 Python 함수를 적용하는 것일 수도 있다.

체인은 세가지 타입으로 분류를 할 수 있다. 유틸리티 체인, 제네릭 체인, 문서 결합(Combine Documents) 체인이다. 이 책에서는 앞의 두 개에 대해서 집중한다. 왜냐하면 문서결합 체인은 좀 특수한 체인이다.

1. 유틸리티 체인 : 특별한/제한적인(very narrow) 목적을 가지고 llm으로부터 특정 답변을 추출해 내기 위한 체인이다. 이미 Langchain내에 만들어져 있어서(out of the box) 곧바로 사용할 수 있다.
2. 제네릭 체인 : 다른 체인들을 엮어(building block) 내는 데 필요한 체인이다. 따라서 곧바로 사용할 수는 없다. 뭔가 엮어내는 코딩을 해야 한다는 뜻이다.

각 체인들이 어떤 기능들을 우리에게 제공하는 지 살펴보자.

### 유틸리티 체인
<b>LLMMathChain</b>라고 하는 아주 간단한 체인을 한 번 살펴보자. LLMMathChain은 산수를 할 수 있는 기능을 llm에게 주는 역할을 한다.

In [10]:
llm_math = LLMMathChain(llm=llm, verbose=True)
count_tokens(llm_math, "What is 13 raised to the .3432 power?")

/Users/minsoojo/Library/Caches/pypoetry/virtualenvs/pinecone-ai-handbook-m0t52s90-py3.10/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?```text
13**0.3432
```
...numexpr.evaluate("13**0.3432")...

Answer: 2.4116004626599237
> Finished chain.
전체 사용 토큰 : 228


'Answer: 2.4116004626599237'

어떤 일이 벌어졌는 지 살펴보자. LLMMathChain은 자연어로된 질문(인풋)을 받아서 llm에게 보냈다. llm은 산수 표현식을 내보냈고, 체인은 이 표현식을 계산하기 위해 numexpr 이라는 python 라이브러리를 사용하는 python 코드를 실행을 시켰다. 여기서 우리는 'llm이 우리의 질문에 답하기 위해 numexpr를 사용해야 한다는 것을 어떻게 알아냈을까?'

#### 프롬프트로 들어가기

체인에게 우리가 인풋으로 보낸 질문만을 llm이 받은 것은 아니다. 인풋은 실제로른 더 확장된 컨텍스트 속으로 들어간다. 컨텍스트 내에서는 좀 더 정확한 인스트럭션이 있어서 우리가 보낸 인풋을 어떻게 해석을 해야 하는 지에 대한 것을 알 수 있다. 우리는 이것을 <b><i>프롬프트</i><b>라고 부른다. LLMMathChain의 프롬프트가 어떠한 지를 찾아내자.

In [11]:
print(llm_math.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



위 템플릿이 핵심이다. 2장에서 본 Few Shot Prompt Template과 같이 예제가 들어 있다. 우리가 문자로 표현된 복잡한 산수 문제를 llm에게 인풋으로 주었을 때 llm은 산수를 직접 수행하지 않는다. 대신 numexpr 라이브러리를 사용해서 답을 하라고 프롬프트가 구성이 되어있다. 아마 우리가 이러한 컨텍스트 없이 인풋을 llm에게 전달하면 이 문제를 풀 수 없을 것이다. 실제로 한 번 해보자.

In [12]:
# 프롬프트를 우리의 인풋만을 통해서 만들어 지도록 구성하자.
prompt = PromptTemplate(input_variables=['question'], template='{question}')
llm_chain = LLMChain(prompt=prompt, llm=llm)

# llm에게 질문한다.
count_tokens(llm_chain, "What is 13 raised to the .3432 power?")

전체 사용 토큰 : 28


'\n\n13 raised to the .3432 power is approximately 2.999.'

잘못된 답을 내놓았다! 지금까지의 설명에 '프롬프트'의 역할이 아주 잘 드러나 있다. 우리는 이러한 인사이트를 얻을 수 있다.
<b>인사이트</b>: 프롬프트를 똑똑하게(intelligently) 사용하면, llm이 특정 방식으로 작동하도록 명시적이고 의도적으로 프로그래밍을 하게 되어 일반적으로 발생하는 llm의 함정을 피할 수 있다.

참고로, 좀 더 깊이 들어가 보면 아래 결과처럼, llm_math의 _call 함수의 소스를 볼 수 있다. 우리가 파이썬 코드를 잘 몰라도 대충 어떻게 실행이 되는 지를 알 수 있다. 질문을 계속 분해를 해서 _process_llm_result 함수에게로 보내고, _process_llm_result에서는 _evaluate_expression 을 호출한다. 여기서는 최종적으로 numexpr.evaluate를 호출하여 계산을 하고 이를 결과로 던진다.

In [13]:
print(inspect.getsource(llm_math._call))

    def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        _run_manager.on_text(inputs[self.input_key])
        llm_output = self.llm_chain.predict(
            question=inputs[self.input_key],
            stop=["```output"],
            callbacks=_run_manager.get_child(),
        )
        return self._process_llm_result(llm_output, _run_manager)



In [14]:
print(inspect.getsource(llm_math._process_llm_result))

    def _process_llm_result(
        self, llm_output: str, run_manager: CallbackManagerForChainRun
    ) -> Dict[str, str]:
        run_manager.on_text(llm_output, color="green", verbose=self.verbose)
        llm_output = llm_output.strip()
        text_match = re.search(r"^```text(.*?)```", llm_output, re.DOTALL)
        if text_match:
            expression = text_match.group(1)
            output = self._evaluate_expression(expression)
            run_manager.on_text("\nAnswer: ", verbose=self.verbose)
            run_manager.on_text(output, color="yellow", verbose=self.verbose)
            answer = "Answer: " + output
        elif llm_output.startswith("Answer:"):
            answer = llm_output
        elif "Answer:" in llm_output:
            answer = "Answer: " + llm_output.split("Answer:")[-1]
        else:
            raise ValueError(f"unknown format from LLM: {llm_output}")
        return {self.output_key: answer}



In [16]:
print(inspect.getsource(llm_math._evaluate_expression))

    def _evaluate_expression(self, expression: str) -> str:
        import numexpr

        try:
            local_dict = {"pi": math.pi, "e": math.e}
            output = str(
                numexpr.evaluate(
                    expression.strip(),
                    global_dict={},  # restrict access to globals
                    local_dict=local_dict,  # add common mathematical functions
                )
            )
        except Exception as e:
            raise ValueError(
                f'LLMMathChain._evaluate("{expression}") raised error: {e}.'
                " Please try again with a valid numerical expression"
            )

        # Remove any leading and trailing brackets from the output
        return re.sub(r"^\[|\]$", "", output)



유틸리티 체인은 이런 식으로 대부분 구성이 되어 있다: 주어진 쿼리(input)에 대해서 답을 하기 위해 아주 특정적인 답을 내도록 llm에게 제한을 가하는 프롬프트가 존재한다. 이런 식으로 SQL 쿼리 문장을 만드는 llm을 만들 수도 있고(이미 만들어져 있다), API 호출을 하거나 리눅스 Bash 명령어를 실행하도록 하는 체인도 있다.
이러한 유틸리티 체인은 계속해서 만들어질 것이며 이미 존재하는 체인은 ![여기](https://api.python.langchain.com/en/v0.1/langchain_api_reference.html#module-langchain.chains)에서 살펴볼 수 있다.

#### 제네릭 체인
Langchain에는 세 개의 제네릭 체인이 있고 예제를 통해 한꺼번에 이를 살펴볼 것이다. 세 개의 체인은 TransformChain, LLMChain, SequentialChain이다.
TransformChain은 말 그대로 인풋을 특정 형태의 아웃풋으로 바꾸는 체인이다. 특정 형태에 대해서는 우리가 함수를 지정할 수가 있다. 먼저 아래와 같이 지저분한 인풋에서 엔터가 여러번 들어가거나 공백이 여러개 연속되어 있는 부분을 하나로 깔끔하게 만드는 함수를 만들자.


In [17]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]

    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text) # 정규표현식을 통해 엔터가 두 번 이상 나오는 경우 한 번으로 바꾼다.
    text = re.sub(r'[ \t]+', ' ', text) # 정규표현식을 사용해 공백 혹은 탭문자가 1개 이상 오는 경우 하나의 공백으로 바꾼다.

    return {"output_text": text}


Transformchain을 초기화 할 때 우리는 llm을 인자로 전달하지 않는다. llm을 인자로 주지 않았기 때문에 우리가 아래에서 만드는 체인은 이전에 본 체인처럼 똑똑하지는 않다. 하지만 이 체인을 다른 체인들과 연결을 할 때 좀 더 좋은 결과를 얻는 것을 보게 될 것이다.

In [20]:
clean_extra_spaces_chain = TransformChain(input_variables=["text"], output_variables=["output_text"], transform=transform_func)

clean_extra_spaces_chain.run('A random text  with    some irregular spacing.\n\n\n     Another one   here as well.')

'A random text with some irregular spacing.\n Another one here as well.'

어떤 경우에는 우리는 인풋 텍스트를 특정 스타일로 만들고 싶을 때가 있다. 예를 들어 인풋 텍스트를 시(poet)의 형태로 바꾸거나, 경찰이 말하는 어투로 바꿔야 할 때가 있다. TransformChain은 llm을 사용하지 않기 때문에 이런 일을 할 수 없다. LLMChain을 사용하면 이런 일을 할 수 있다.

먼저 프롬프트 템플릿을 만들자.

In [21]:
template = """Paraphrase this text:

{output_text}

In the style of a {style}.

Paraphrase: """

prompt = PromptTemplate(input_variables=["style","output_text"], template=template)

style_paraphrase_chain = LLMChain(llm=llm, prompt=prompt, output_key='final_output')

위에서 보면 템플릿에서 인풋 텍스트가 와야할 자리의 변수에 'output_text'라고 되어 있는 것을 알 수 있다. 왜그럴까? 이유는 TransformChain의 아웃풋이 인풋으로 들어갈 것이기 때문에 TransformChain의 output_text가 템플릿의 인풋 텍스트가 되는 것이다.

마지막으로 우리는 이 두 개의 체인을 하나로 엮을 것이다. 이를 위해 SequentialChain을 사용한다.

In [22]:
sequential_chain = SequentialChain(
    chains=[clean_extra_spaces_chain, style_paraphrase_chain], 
    input_variables=['text', 'style'],
    output_variables=['final_output'])

우리가 예제로 사용할 인풋은 langchain 공식 문서에 나와 있는 chain이란 무엇인가를 설명하는 부분인데 필요 없는 공백들이 들어 있는 문장이다.

In [23]:
input_text = """
Chains allow us to combine multiple 


components together to create a single, coherent application. 

For example, we can create a chain that takes user input,       format it with a PromptTemplate, 

and then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by 


combining chains with other components.
"""

마지막으로 한 번 실행을 해보자. 아래의 결과 문장이 90년대 래퍼 스타일의 문장인가?

In [24]:
count_tokens(sequential_chain, {'text':input_text, 'style': 'a 90s rapper'})

전체 사용 토큰 : 165


"Yo, check it - chains be the key to makin' one dope app. We can link up different parts, like takin' user input and mixin' it with a PromptTemplate, then passin' it to an LLM. And we can get even crazier by mixin' multiple chains or mixin' chains with other fly components."